# Imports

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


import os
import sys
import torch
from tqdm import tqdm
import pandas as pd
import random
# import copy
# import h5py
# import numpy as np
# import pandas as pd
from IPython.display import display
# import random
# import matplotlib.ticker as ticker
# import matplotlib.pyplot as plt
import numpy as np
# import itertools
# import sklearn


# local imports
sys.path.insert(
    0,
    os.path.join(
        os.path.dirname(os.path.abspath(''))
    )
)
from dataloaders.cifar10 import (
    load_cifar10
)
from dataloaders.cifar100 import (
    load_cifar100
)
from models import models_dict as MODELS_DICT
# models_dict = {
#     "resnet18_cifar": resnet18_cifar,
#     "resnet18_torch": torchvision.models.resnet18,
#     "resnet50_torch": torchvision.models.resnet50,
#     "densenet_cifar": densenet_cifar
# }
sys.path.pop(0)


'/scratch_local/arubinstein17-163577/starlight'

# Functions

In [3]:

def apply_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"  # to suppress warning
    torch.use_deterministic_algorithms(True, warn_only=True)


def extract_list_from_huge_string(huge_string):
    assert isinstance(huge_string, str)
    split = huge_string.split('\n')
    res = []
    for el in split:
        el = el.strip()
        if el != '':
            res.append(el)
    return res


def get_model_id(model_path):
    return model_path.split(os.sep)[-1].split(".")[0]


def get_link(raw_link):
    id = raw_link.split("/d/")[-1].split("/")[0]
    return f"https://drive.google.com/uc?id={id}"


def get_model(chkpt_path, model_name, model_settings={}):
    model = MODELS_DICT[model_name](**model_settings)
    model.load_state_dict(torch.load(chkpt_path)["state_dict"])
    return model


def get_resnet18_cifar(chkpt_path, num_classes=10):
    return get_model(
        chkpt_path,
        "resnet18_cifar",
        model_settings={"num_classes": num_classes}
)


def get_resnet18_cifar100(chkpt_path):
    return get_resnet18_cifar(chkpt_path, num_classes=100)


def get_paths(root_path):
    stars_path = os.path.join(root_path, "stars")
    anchors_path = os.path.join(root_path, "anchors")

    anchors = [
        os.path.join(anchors_path, name) for name in os.listdir(anchors_path)
    ]
    stars = [
        os.path.join(stars_path, name) for name in os.listdir(stars_path)
    ]
    return anchors, stars


@torch.no_grad
def compute_margin_for_batch(model, x, device="cuda:0", agg="mean", norm="l1"):

    x.requires_grad = True

    with torch.enable_grad():
        output = model(x)

    output = output.to(device)

    num_classes = output.shape[-1]
    batch_size = output.shape[0]
    used_classes = range(num_classes)

    model_grads = {}

    with torch.enable_grad():
        for i, class_i in enumerate(used_classes):
            model_grads[class_i] = get_grad(
                output[..., class_i],
                x,
                retain_graph=(i != len(used_classes) - 1)
            )

        if agg == "min_to_pred":
            margins_for_all_class_pairs = torch.ones(
                batch_size,
                num_classes,
                num_classes
            ).to(output.device) * torch.inf
        else:
            margins_for_all_class_pairs = torch.zeros(
                batch_size,
                num_classes,
                num_classes
            ).to(output.device)

        for i in range(len(used_classes)):
            for j in range(i + 1, len(used_classes)):

                class_i = used_classes[i]
                class_j = used_classes[j]

                output_i = output[..., class_i]
                output_j = output[..., class_j]

                grad_i = model_grads[class_i]
                grad_j = model_grads[class_j]
                batch_margins_for_class_pair = compute_margin(
                    output_i,
                    output_j,
                    grad_i,
                    grad_j,
                    norm=norm
                )

                # [batch_size, num_classes, num_classes],
                # (s, i, j) - margin between classes i and j for sample s
                margins_for_all_class_pairs[:, class_i, class_j] \
                    = batch_margins_for_class_pair
                margins_for_all_class_pairs[:, class_j, class_i] \
                    = batch_margins_for_class_pair

        if agg == "min_to_pred":
            pred = output.argmax(dim=-1)

            min_distances_for_all_classes = margins_for_all_class_pairs.min(dim=-1).values

            min_distances_to_pred = min_distances_for_all_classes[
                torch.arange(batch_size).to(pred.device),
                pred
            ]
            return min_distances_to_pred.mean()
        elif agg == "max_first":
            # sanity check that it is higher than min_to_pred
            res = margins_for_all_class_pairs.max(dim=-1).values.max(-1).values.mean()
            return res
        else:
            res = margins_for_all_class_pairs.mean(dim=0)
            if agg == "mean":
                # diagonal for class_i == class_j has only zeros,
                # so we don't need to use them in mean computation
                return res.mean() * (num_classes / (num_classes - 1))
            else:
                assert agg == "max"
                return res.max()


def get_grad(output_i, x, retain_graph):
    assert x.requires_grad == True
    differentiable_value = output_i.sum()
    differentiable_value.backward(inputs=x, retain_graph=retain_graph)
    grad = x.grad
    x.grad = None
    return grad


def compute_margin(output_i, output_j, grad_i, grad_j, norm="l1"):
    # eq. 7 here: https://proceedings.neurips.cc/paper_files/paper/2018/file/42998cf32d552343bc8e460416382dca-Paper.pdf
    # d = |f_i(x) - f_j(x)| / |grad_x f_i(x) - grad_x f_j (x)|_q
    outputs_diff = (output_i - output_j).abs()
    grads_diff = (grad_i - grad_j)
    if norm == "l1":
        return (
                outputs_diff
            /
                grads_diff.abs().sum()
        )
    elif norm == "l2":
        return (
                outputs_diff
            /
                # grads_diff.abs().pow(2).sum().sqrt()
                torch.linalg.vector_norm(grads_diff, ord=2)
        )
    else:
        raise NotImplementedError("Only L1 norm is supported")


def margin_on_dataloader(
    model,
    dataloader,
    percent=0.01,
    device="cuda:0",
    agg="mean",
    norm="l1",
    random_seed=42
):

    model.eval()
    model.to(device)

    dataloader_len = len(dataloader)
    if percent == 1:
        active_batches = list(range(dataloader_len))
    else:
        total_batches = int(percent * dataloader_len)
        active_batches = np.linspace(0, dataloader_len, total_batches)
        active_batches = active_batches.astype(int)

    res = []
    pb = tqdm(total=dataloader_len, desc='batch', position=0)
    apply_random_seed(random_seed)
    for i, batch in (enumerate(dataloader)):
        pb.update(1)
        if i not in active_batches:
            continue
        x, y = batch
        x = x.to(device)
        batch_margin = compute_margin_for_batch(
            model,
            x,
            device=device,
            agg=agg,
            norm=norm
        )
        res.append(batch_margin)
    return torch.Tensor(res).mean().item()


def compute_results(
    model_paths,
    dataloader,
    cache_path,
    split_name,
    percent=1,
    load_model=get_resnet18_cifar,
    recompute_substring=None,
    agg="mean",
    norm="l1"
):
    res = {}
    if os.path.exists(cache_path):
        print(f"Loading existing results from {cache_path}")
        res = torch.load(cache_path)
    else:
        res = {}

    dataset_root = dataloader.dataset.dataset.__dict__["root"]
    dataset_name = dataset_root.split(os.sep)[-1]
    for model_path in model_paths:

        path_split = model_path.split(os.sep)[-2:]
        assert len(path_split) == 2
        model_type = path_split[0]
        model_id = path_split[1].split(".")[0]
        df_id = f"{model_type}_{model_id}_{dataset_name}_p{percent}"
        to_recompute = (not df_id in res)
        if not to_recompute and recompute_substring is not None:
            to_recompute = recompute_substring in df_id

        if to_recompute:
            print(f"Computing results for {df_id}")
            res[df_id] = {}
            res[df_id]["margin"] = margin_on_dataloader(
                load_model(model_path),
                dataloader,
                percent=percent,
                device="cuda:0",
                agg=agg,
                norm=norm
            )
            res[df_id]["model_type"] = model_type
            res[df_id]["dataset_name"] = dataset_name
            res[df_id]["percent"] = percent
            res[df_id]["model_id"] = model_id
            res[df_id]["split_name"] = split_name
        else:
            print(f"Reusing results for {df_id}")
    torch.save(res, cache_path)
    return res


def plot_dict(
    res,
    to_show=True,
    filter_dict={},
    drop_columns=[],
    group_aggregate_list=[],
    query=None
):

    df = pd.DataFrame.from_dict(res, orient='index')

    for column_name, allowed_values in filter_dict.items():
        df = df[df[column_name] in allowed_values]

    if query is not None:
        df = df.query(query)

    for column_name in drop_columns:
        if column_name == "index":
            df.reset_index(drop=True, inplace=True)
        else:
            df = df.drop(column_name, axis=1)

    for new_index in group_aggregate_list:
        df = df.groupby(new_index).agg([(f'mean', 'mean'), (f'std', 'std')])

    if to_show:
        display(df)
    return df

# Prepare cifar100

In [177]:
get_link("https://drive.google.com/file/d/1zTH5_YBM9h4AYfeQWhkJyqXW_14xS38B/view?usp=drive_link")

'https://drive.google.com/uc?id=1zTH5_YBM9h4AYfeQWhkJyqXW_14xS38B'

In [178]:
!gdown https://drive.google.com/uc?id=1zTH5_YBM9h4AYfeQWhkJyqXW_14xS38B

Downloading...
From (original): https://drive.google.com/uc?id=1zTH5_YBM9h4AYfeQWhkJyqXW_14xS38B
From (redirected): https://drive.google.com/uc?id=1zTH5_YBM9h4AYfeQWhkJyqXW_14xS38B&confirm=t&uuid=51724791-d6b0-4e41-9a83-bae8a9e60ecc
To: /scratch_local/arubinstein17-163577/starlight/notebooks/cifar100_resnet18.zip
100%|██████████████████████████████████████| 4.38G/4.38G [00:57<00:00, 76.1MB/s]


In [182]:
!unzip cifar100_resnet18.zip -d ./

Archive:  cifar100_resnet18.zip
   creating: ./cifar100_resnet18/
   creating: ./cifar100_resnet18/anchors/
  inflating: ./cifar100_resnet18/anchors/0eqk0gla.pt  
  inflating: ./cifar100_resnet18/anchors/384qnuxm.pt  
  inflating: ./cifar100_resnet18/anchors/3n8ftowe.pt  
  inflating: ./cifar100_resnet18/anchors/4808726s.pt  
  inflating: ./cifar100_resnet18/anchors/4rjdi4pa.pt  
  inflating: ./cifar100_resnet18/anchors/5bfkmro6.pt  
  inflating: ./cifar100_resnet18/anchors/65cr1zyr.pt  
  inflating: ./cifar100_resnet18/anchors/6hapim4a.pt  
  inflating: ./cifar100_resnet18/anchors/73s47o0m.pt  
  inflating: ./cifar100_resnet18/anchors/75eeo9bw.pt  
  inflating: ./cifar100_resnet18/anchors/7wgab2a1.pt  
  inflating: ./cifar100_resnet18/anchors/82yrfx4p.pt  
  inflating: ./cifar100_resnet18/anchors/85izljav.pt  
  inflating: ./cifar100_resnet18/anchors/8pppxxoh.pt  
  inflating: ./cifar100_resnet18/anchors/9ansq593.pt  
  inflating: ./cifar100_resnet18/anchors/9lzove03.pt  
  inflating:

In [4]:
last_10_stars_cifar100_resnet18_names = extract_list_from_huge_string(
    """212q5op8
ly0u9r9m
t200saru
zcgviume
mgcabk97
2umwboaq
wa8x6rt9
yvi97vmj
lgd1aief
rylbd95p"""
)

In [5]:
anchors_cifar100_resnet18, stars_cifar100_resnet18 = get_paths("./cifar100_resnet18/")

cifar100_train_dataloader, _, _ = load_cifar100(
    batch_size=256,
    num_workers=8,
    img_size=32,
    normalize=True,
    resize=False,
    horizontal_flip=False,
    vertical_flip=False,
    random_crop_resize=False,
    random_resize_crop=False,
    color_jitter=False,
    rotation_range=0,
    pad_random_crop=False,
    random_one_aug=False,
    train_set_fraction=1.0,
    return_ds=False,
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Using 45000 images for training
Using 5000 images for validation
Using train transform Compose(
    IdentityTransform()
    ToTensor()
    Normalize(mean=[0.485 0.456 0.406], std=[0.229 0.224 0.225])
)


# Prepare cifar10

In [ ]:
get_link("https://drive.google.com/file/d/1g-TxEGbORtHmxVEefoJtk2yxSf_mHL28/view?usp=drive_link")

In [ ]:
!gdown https://drive.google.com/uc?id=1g-TxEGbORtHmxVEefoJtk2yxSf_mHL28

In [ ]:
!unzip cifar10_resnet18.zip -d ./

In [4]:
last_10_stars = extract_list_from_huge_string("""8dirowph
wrvljast
nyolmhfc
82pxe1i8
n8vpmpbj
h2qjwxrq
n5sjdrla
6gps5q6c
bezgu0h7
by2vpp9d""")

In [5]:
anchors_resnet18, stars_resnet18 = get_paths("./cifar10_resnet18/")

# stars_resnet18 = [path for path in stars_resnet18 if get_model_id(path) in last_10_stars]

cifar10_train_dataloader, _, _ = load_cifar10(
    batch_size=256,
    num_workers=8,
    img_size=32,
    normalize=True,
    resize=False,
    horizontal_flip=False,
    vertical_flip=False,
    random_crop_resize=False,
    random_resize_crop=False,
    color_jitter=False,
    rotation_range=0,
    pad_random_crop=False,
    random_one_aug=False,
    train_set_fraction=1.0,
    return_ds=False,
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Using 45000 images for training
Using 5000 images for validation
Using train transform Compose(
    IdentityTransform()
    ToTensor()
    Normalize(mean=[0.485 0.456 0.406], std=[0.229 0.224 0.225])
)


# Compute margin

## Resnet18 Cifar100

### L2 + min to pred

In [6]:
cifar100_resnet18_res_path = "./min2pred_l2_cifar100_resnet18_results.pt"

last_10_stars_cifar100_resnet18 = [
    path for path in stars_cifar100_resnet18 if get_model_id(path) in last_10_stars_cifar100_resnet18_names
]

max_cifar100_resnet18_results = compute_results(
    anchors_cifar100_resnet18[:10] + last_10_stars_cifar100_resnet18,
    cifar100_train_dataloader,
    cifar100_resnet18_res_path,
    split_name="train",
    percent=0.1,
    load_model=get_resnet18_cifar100,
    # recompute_substring="p0"
    recompute_substring=None,
    agg="min_to_pred",
    norm="l2"
)

Computing results for anchors_0eqk0gla_cifar100_p0.1


batch: 100%|██████████| 176/176 [01:29<00:00,  1.96it/s]


Computing results for anchors_384qnuxm_cifar100_p0.1


batch: 100%|██████████| 176/176 [01:30<00:00,  1.95it/s]


Computing results for anchors_3n8ftowe_cifar100_p0.1


...

batch: 100%|██████████| 176/176 [01:30<00:00,  1.95it/s]


Computing results for stars_yvi97vmj_cifar100_p0.1


batch: 100%|██████████| 176/176 [01:30<00:00,  1.95it/s]


Computing results for stars_zcgviume_cifar100_p0.1


batch: 100%|██████████| 176/176 [01:30<00:00,  1.95it/s]


In [8]:
df = plot_dict(max_cifar100_resnet18_results, to_show=True)

,margin,model_type,dataset_name,percent,model_id,split_name
anchors_0eqk0gla_cifar100_p0.1,0.104104,anchors,cifar100,0.1,0eqk0gla,train
anchors_384qnuxm_cifar100_p0.1,0.107184,anchors,cifar100,0.1,384qnuxm,train
anchors_3n8ftowe_cifar100_p0.1,0.100297,anchors,cifar100,0.1,3n8ftowe,train
anchors_4808726s_cifar100_p0.1,0.106649,anchors,cifar100,0.1,4808726s,train
anchors_4rjdi4pa_cifar100_p0.1,0.101742,anchors,cifar100,0.1,4rjdi4pa,train
anchors_5bfkmro6_cifar100_p0.1,0.101090,anchors,cifar100,0.1,5bfkmro6,train
anchors_65cr1zyr_cifar100_p0.1,0.099083,anchors,cifar100,0.1,65cr1zyr,train
anchors_6hapim4a_cifar100_p0.1,0.103731,anchors,cifar100,0.1,6hapim4a,train
anchors_73s47o0m_cifar100_p0.1,0.098944,anchors,cifar100,0.1,73s47o0m,train
anchors_75eeo9bw_cifar100_p0.1,0.102124,anchors,cifar100,0.1,75eeo9bw,train


In [9]:
df = plot_dict(max_cifar100_resnet18_results, to_show=True, filter_dict={}, drop_columns=["index", "dataset_name", "model_id", "split_name"], group_aggregate_list=["model_type"], query="percent == 0.10")

margin           percent     
                mean       std    mean  std
model_type                                 
anchors     0.102495  0.002889     0.1  0.0
stars       0.066660  0.000641     0.1  0.0

### Max

In [259]:
cifar100_resnet18_res_path = "./max_cifar100_resnet18_results.pt"

last_10_stars_cifar100_resnet18 = [
    path for path in stars_cifar100_resnet18 if get_model_id(path) in last_10_stars_cifar100_resnet18_names
]

max_cifar100_resnet18_results = compute_results(
    anchors_cifar100_resnet18[:10] + last_10_stars_cifar100_resnet18,
    cifar100_train_dataloader,
    cifar100_resnet18_res_path,
    split_name="train",
    percent=0.1,
    load_model=get_resnet18_cifar100,
    # recompute_substring="p0"
    recompute_substring=None,
    agg="max"
)

Computing results for anchors_0eqk0gla_cifar100_p0.1


batch: 100%|██████████| 176/176 [00:49<00:00,  3.55it/s]


Computing results for anchors_384qnuxm_cifar100_p0.1


batch: 100%|██████████| 176/176 [00:49<00:00,  3.55it/s]


Computing results for anchors_3n8ftowe_cifar100_p0.1


...

batch: 100%|██████████| 176/176 [00:49<00:00,  3.54it/s]


Computing results for stars_yvi97vmj_cifar100_p0.1


batch: 100%|██████████| 176/176 [00:49<00:00,  3.54it/s]


Computing results for stars_zcgviume_cifar100_p0.1


batch: 100%|██████████| 176/176 [00:49<00:00,  3.54it/s]


In [260]:
df = plot_dict(max_cifar100_resnet18_results, to_show=True, filter_dict={}, drop_columns=["index", "dataset_name", "model_id", "split_name"], group_aggregate_list=["model_type"], query="percent == 0.10")

margin               percent     
                mean           std    mean  std
model_type                                     
anchors     0.000046  9.583485e-07     0.1  0.0
stars       0.000043  6.390722e-07     0.1  0.0

### Mean

In [240]:
cifar100_resnet18_res_path = "./cifar100_resnet18_results.pt"

last_10_stars_cifar100_resnet18 = [
    path for path in stars_cifar100_resnet18 if get_model_id(path) in last_10_stars_cifar100_resnet18_names
]

cifar100_resnet18_results = compute_results(
    anchors_cifar100_resnet18[:10] + last_10_stars_cifar100_resnet18,
    cifar100_train_dataloader,
    cifar100_resnet18_res_path,
    split_name="train",
    percent=0.1,
    load_model=get_resnet18_cifar100,
    # recompute_substring="p0"
    recompute_substring=None
)

Loading existing results from ./cifar100_resnet18_results.pt
Reusing results for anchors_0eqk0gla_cifar100_p0.1
Reusing results for anchors_384qnuxm_cifar100_p0.1
Reusing results for anchors_3n8ftowe_cifar100_p0.1
Reusing results for anchors_4808726s_cifar100_p0.1
Reusing results for anchors_4rjdi4pa_cifar100_p0.1
Reusing results for anchors_5bfkmro6_cifar100_p0.1
Reusing results for anchors_65cr1zyr_cifar100_p0.1
Reusing results for anchors_6hapim4a_cifar100_p0.1
Reusing results for anchors_73s47o0m_cifar100_p0.1
Reusing results for anchors_75eeo9bw_cifar100_p0.1
Computing results for stars_212q5op8_cifar100_p0.1


batch: 100%|██████████| 176/176 [00:49<00:00,  3.58it/s]


Computing results for stars_2umwboaq_cifar100_p0.1


batch: 100%|██████████| 176/176 [00:49<00:00,  3.56it/s]


Computing results for stars_lgd1aief_cifar100_p0.1


...

batch: 100%|██████████| 176/176 [00:49<00:00,  3.55it/s]


Computing results for stars_yvi97vmj_cifar100_p0.1


batch: 100%|██████████| 176/176 [00:49<00:00,  3.55it/s]


Computing results for stars_zcgviume_cifar100_p0.1


batch: 100%|██████████| 176/176 [00:49<00:00,  3.56it/s]


In [252]:
df = plot_dict(cifar100_resnet18_results, to_show=True)

,margin,model_type,dataset_name,percent,model_id,split_name
anchors_0eqk0gla_cifar100_p0.05,0.000035,anchors,cifar100,0.05,NaN,NaN
anchors_384qnuxm_cifar100_p0.05,0.000035,anchors,cifar100,0.05,NaN,NaN
stars_01fj1cav_cifar100_p0.05,0.000034,stars,cifar100,0.05,NaN,NaN
stars_212q5op8_cifar100_p0.05,0.000033,stars,cifar100,0.05,NaN,NaN
anchors_0eqk0gla_cifar100_p0.1,0.000034,anchors,cifar100,0.10,0eqk0gla,train
anchors_384qnuxm_cifar100_p0.1,0.000035,anchors,cifar100,0.10,384qnuxm,train
anchors_3n8ftowe_cifar100_p0.1,0.000033,anchors,cifar100,0.10,3n8ftowe,train
anchors_4808726s_cifar100_p0.1,0.000035,anchors,cifar100,0.10,4808726s,train
anchors_4rjdi4pa_cifar100_p0.1,0.000034,anchors,cifar100,0.10,4rjdi4pa,train
anchors_5bfkmro6_cifar100_p0.1,0.000034,anchors,cifar100,0.10,5bfkmro6,train


In [ ]:
df = plot_dict(cifar100_resnet18_results, to_show=True, filter_dict={}, drop_columns=["index", "dataset_name"], group_aggregate_list=["model_type"])

In [251]:
df = plot_dict(cifar100_resnet18_results, to_show=True, filter_dict={}, drop_columns=["index", "dataset_name", "model_id", "split_name"], group_aggregate_list=["model_type"], query="percent == 0.10")

margin               percent     
                mean           std    mean  std
model_type                                     
anchors     0.000034  7.466623e-07     0.1  0.0
stars       0.000033  2.407625e-07     0.1  0.0

## Resnet18 Cifar10

### L2 + min margin to pred

In [92]:
cifar10_resnet18_res_path = "./min2pred_l2_cifar10_resnet18_results.pt"

last_10_stars_cifar10_resnet18 = [path for path in stars_resnet18 if get_model_id(path) in last_10_stars]

max_cifar10_resnet18_results = compute_results(
    anchors_resnet18[:10] + last_10_stars_cifar10_resnet18,
    # anchors_resnet18[:2] + last_10_stars_cifar10_resnet18[:2],
    cifar10_train_dataloader,
    cifar10_resnet18_res_path,
    split_name="train",
    percent=1,
    # percent=0.05,
    load_model=get_resnet18_cifar,
    # recompute_substring="p0"
    recompute_substring=None,
    agg="min_to_pred",
    norm="l2"
)

Computing results for anchors_2oo0upw1_cifar10_p1


batch:   1%|          | 1/176 [00:00<00:41,  4.23it/s]/scratch_local/arubinstein17-163577/starlight/starlight_env/lib/python3.9/site-packages/torch/autograd/__init__.py:266: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:164.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
batch: 100%|██████████| 176/176 [01:31<00:00,  1.92it/s]


Computing results for anchors_3m2c3guk_cifar10_p1


batch: 100%|██████████| 176/176 [01:32<00:00,  1.90it/s]


Computing results for anchors_448kfne2_cifar10_p1


...

batch: 100%|██████████| 176/176 [01:32<00:00,  1.90it/s]


Computing results for stars_nyolmhfc_cifar10_p1


batch: 100%|██████████| 176/176 [01:32<00:00,  1.90it/s]


Computing results for stars_wrvljast_cifar10_p1


batch: 100%|██████████| 176/176 [01:32<00:00,  1.90it/s]


In [95]:
df = plot_dict(max_cifar10_resnet18_results, to_show=True)

,margin,model_type,dataset_name,percent,model_id,split_name
anchors_2oo0upw1_cifar10_p1,0.118189,anchors,cifar10,1,2oo0upw1,train
anchors_3m2c3guk_cifar10_p1,0.118039,anchors,cifar10,1,3m2c3guk,train
anchors_448kfne2_cifar10_p1,0.112324,anchors,cifar10,1,448kfne2,train
anchors_52l22gc1_cifar10_p1,0.118346,anchors,cifar10,1,52l22gc1,train
anchors_5791j5is_cifar10_p1,0.116815,anchors,cifar10,1,5791j5is,train
anchors_5hrwt8sd_cifar10_p1,0.121371,anchors,cifar10,1,5hrwt8sd,train
anchors_6ids7w54_cifar10_p1,0.108609,anchors,cifar10,1,6ids7w54,train
anchors_6kxd228s_cifar10_p1,0.112535,anchors,cifar10,1,6kxd228s,train
anchors_7sckybft_cifar10_p1,0.120158,anchors,cifar10,1,7sckybft,train
anchors_7y4wdzel_cifar10_p1,0.122731,anchors,cifar10,1,7y4wdzel,train


In [97]:
df = plot_dict(max_cifar10_resnet18_results, to_show=True, drop_columns=["index", "dataset_name", "model_id", "split_name"], group_aggregate_list=["model_type"], query="percent == 1.00")

margin           percent     
                mean       std    mean  std
model_type                                 
anchors     0.116912  0.004452     1.0  0.0
stars       0.097228  0.002544     1.0  0.0

### Max

In [258]:
cifar10_resnet18_res_path = "./max_cifar10_resnet18_results.pt"

last_10_stars_cifar10_resnet18 = [path for path in stars_resnet18 if get_model_id(path) in last_10_stars]

max_cifar10_resnet18_results = compute_results(
    anchors_resnet18[:10] + last_10_stars_cifar10_resnet18,
    # anchors_resnet18[:2] + last_10_stars_cifar10_resnet18[:2],
    cifar10_train_dataloader,
    cifar10_resnet18_res_path,
    split_name="train",
    percent=1,
    # percent=0.05,
    load_model=get_resnet18_cifar,
    # recompute_substring="p0"
    recompute_substring=None,
    agg="max"
)

Loading existing results from ./max_cifar10_resnet18_results.pt
Computing results for anchors_2oo0upw1_cifar10_p1


batch: 100%|██████████| 176/176 [00:51<00:00,  3.41it/s]


Computing results for anchors_3m2c3guk_cifar10_p1


batch: 100%|██████████| 176/176 [00:51<00:00,  3.39it/s]


Computing results for anchors_448kfne2_cifar10_p1


...

batch: 100%|██████████| 176/176 [00:51<00:00,  3.39it/s]


Computing results for stars_nyolmhfc_cifar10_p1


batch: 100%|██████████| 176/176 [00:51<00:00,  3.39it/s]


Computing results for stars_wrvljast_cifar10_p1


batch: 100%|██████████| 176/176 [00:51<00:00,  3.39it/s]


In [23]:
max_cifar10_resnet18_res_path = "./max_cifar10_resnet18_results.pt"
max_cifar10_resnet18_results = torch.load(max_cifar10_resnet18_res_path)

In [24]:
df = plot_dict(max_cifar10_resnet18_results, to_show=True)

,margin,model_type,dataset_name,percent,model_id,split_name
anchors_2oo0upw1_cifar10_p0.05,0.000104,anchors,cifar10,0.05,2oo0upw1,train
anchors_3m2c3guk_cifar10_p0.05,0.000104,anchors,cifar10,0.05,3m2c3guk,train
stars_6gps5q6c_cifar10_p0.05,0.000090,stars,cifar10,0.05,6gps5q6c,train
stars_82pxe1i8_cifar10_p0.05,0.000089,stars,cifar10,0.05,82pxe1i8,train
anchors_2oo0upw1_cifar10_p1,0.000105,anchors,cifar10,1.00,2oo0upw1,train
anchors_3m2c3guk_cifar10_p1,0.000104,anchors,cifar10,1.00,3m2c3guk,train
anchors_448kfne2_cifar10_p1,0.000100,anchors,cifar10,1.00,448kfne2,train
anchors_52l22gc1_cifar10_p1,0.000102,anchors,cifar10,1.00,52l22gc1,train
anchors_5791j5is_cifar10_p1,0.000104,anchors,cifar10,1.00,5791j5is,train
anchors_5hrwt8sd_cifar10_p1,0.000104,anchors,cifar10,1.00,5hrwt8sd,train


In [257]:
df = plot_dict(max_cifar10_resnet18_results, to_show=True, drop_columns=["index", "dataset_name", "model_id", "split_name"], group_aggregate_list=["model_type"], query="percent == 1.00")
# df = plot_dict(max_cifar10_resnet18_results, to_show=True, drop_columns=["index", "dataset_name", "model_id", "split_name"], group_aggregate_list=["model_type"], query="percent == 0.05")

margin               percent     
                mean           std    mean  std
model_type                                     
anchors     0.000104  7.450299e-08    0.05  0.0
stars       0.000090  4.076185e-07    0.05  0.0

### Mean

In [ ]:
margin_on_dataloader(get_resnet18_cifar(anchors_resnet18[0]), get_resnet18_cifar(anchors_resnet18[1]), cifar10_train_dataloader, percent=0.10, device="cuda:0")

In [236]:
cifar10_resnet18_res_path = "./cifar10_resnet18_results.pt"

last_10_stars_cifar10_resnet18 = [path for path in stars_resnet18 if get_model_id(path) in last_10_stars]

cifar10_resnet18_results = compute_results(
    anchors_resnet18[:10] + last_10_stars_cifar10_resnet18,
    cifar10_train_dataloader,
    cifar10_resnet18_res_path,
    split_name="train",
    percent=1,
    load_model=get_resnet18_cifar,
    # recompute_substring="p0"
    recompute_substring=None
)

Loading existing results from ./cifar10_resnet18_results.pt
Computing results for anchors_2oo0upw1_cifar10_p1


batch: 100%|██████████| 176/176 [00:51<00:00,  3.41it/s]


Computing results for anchors_3m2c3guk_cifar10_p1


batch: 100%|██████████| 176/176 [00:51<00:00,  3.40it/s]


Computing results for anchors_448kfne2_cifar10_p1


...

batch: 100%|██████████| 176/176 [00:51<00:00,  3.39it/s]


Computing results for stars_nyolmhfc_cifar10_p1


batch: 100%|██████████| 176/176 [00:51<00:00,  3.39it/s]


Computing results for stars_wrvljast_cifar10_p1


batch: 100%|██████████| 176/176 [00:51<00:00,  3.39it/s]


In [241]:
df = plot_dict(cifar10_resnet18_results, to_show=True)

,margin,model_type,dataset_name,percent,model_id,split_name
anchors_2oo0upw1_cifar10_p0.05,0.000081,anchors,cifar10,0.05,2oo0upw1,NaN
anchors_3m2c3guk_cifar10_p0.05,0.000080,anchors,cifar10,0.05,3m2c3guk,NaN
stars_0eiwx0xx_cifar10_p0.05,0.000071,stars,cifar10,0.05,0eiwx0xx,NaN
stars_4s24tjof_cifar10_p0.05,0.000070,stars,cifar10,0.05,4s24tjof,NaN
anchors_2oo0upw1_cifar10_p1,0.000082,anchors,cifar10,1.00,2oo0upw1,train
anchors_3m2c3guk_cifar10_p1,0.000081,anchors,cifar10,1.00,3m2c3guk,train
anchors_448kfne2_cifar10_p1,0.000077,anchors,cifar10,1.00,448kfne2,train
anchors_52l22gc1_cifar10_p1,0.000080,anchors,cifar10,1.00,52l22gc1,train
anchors_5791j5is_cifar10_p1,0.000081,anchors,cifar10,1.00,5791j5is,train
anchors_5hrwt8sd_cifar10_p1,0.000083,anchors,cifar10,1.00,5hrwt8sd,train


In [246]:
df = plot_dict(cifar10_resnet18_results, to_show=True, drop_columns=["index", "dataset_name", "model_id", "split_name"], group_aggregate_list=["model_type"], query="percent == 1.00")

margin           percent     
                mean       std    mean  std
model_type                                 
anchors     0.000081  0.000003     1.0  0.0
stars       0.000070  0.000002     1.0  0.0

### For battle test

In [89]:
cifar10_resnet18_res_path = "./battle_test_cifar10_resnet18_results.pt"

last_10_stars_cifar10_resnet18 = [path for path in stars_resnet18 if get_model_id(path) in last_10_stars]

cifar10_resnet18_results = compute_results(
    anchors_resnet18[:2] + stars_resnet18[:2],
    cifar10_train_dataloader,
    cifar10_resnet18_res_path,
    split_name="train",
    percent=0.05,
    load_model=get_resnet18_cifar,
    recompute_substring="p0",
    # recompute_substring=None,
    # agg="max"
    # agg="min_to_pred"
    agg="max_first"
)

Loading existing results from ./battle_test_cifar10_resnet18_results.pt
Computing results for anchors_2oo0upw1_cifar10_p0.05


batch:   1%|          | 1/176 [00:00<00:39,  4.43it/s]/scratch_local/arubinstein17-163577/starlight/starlight_env/lib/python3.9/site-packages/torch/autograd/__init__.py:266: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:164.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
batch: 100%|██████████| 176/176 [00:04<00:00, 36.41it/s]


Computing results for anchors_3m2c3guk_cifar10_p0.05


batch: 100%|██████████| 176/176 [00:04<00:00, 36.85it/s]


Computing results for stars_0eiwx0xx_cifar10_p0.05


batch: 100%|██████████| 176/176 [00:05<00:00, 34.84it/s]


Computing results for stars_4s24tjof_cifar10_p0.05


batch: 100%|██████████| 176/176 [00:04<00:00, 36.62it/s]


In [90]:
df = plot_dict(cifar10_resnet18_results, to_show=True)

,margin,model_type,dataset_name,percent,model_id,split_name
anchors_2oo0upw1_cifar10_p0.05,0.000357,anchors,cifar10,0.05,2oo0upw1,train
anchors_3m2c3guk_cifar10_p0.05,0.000352,anchors,cifar10,0.05,3m2c3guk,train
stars_0eiwx0xx_cifar10_p0.05,0.000305,stars,cifar10,0.05,0eiwx0xx,train
stars_4s24tjof_cifar10_p0.05,0.000296,stars,cifar10,0.05,4s24tjof,train


In [8]:
df = plot_dict(cifar10_resnet18_results, to_show=True, drop_columns=["index", "dataset_name", "model_id", "train"], group_aggregate_list=["model_type"], query="model_type == 'anchors' | model_id in ['0eiwx0xx']")

KeyError: "['train'] not found in axis"

In [209]:
# df = plot_dict(cifar10_resnet18_results, to_show=True, drop_columns=["index", "dataset_name", "model_id"], group_aggregate_list=["model_type"], query="model_type == 'anchors' | model_id in ['0eiwx0xx']")

margin               percent     
                mean           std    mean  std
model_type                                     
anchors     0.000081  7.595591e-07    0.05  0.0
stars       0.000071           NaN    0.05  NaN

# Save df

In [10]:
df.to_csv("tmp.csv")